<a href="https://colab.research.google.com/github/RaisyaPuspa23/kriptografi/blob/main/Tugas_2_Algoritma_Modern_(RSA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycryptodome > /dev/null

from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
from Crypto.Signature import pss
from Crypto.Hash import SHA256
import hashlib

In [2]:
def aes_encrypt(data):
    key = get_random_bytes(16)
    cipher = AES.new(key, AES.MODE_GCM)
    ciphertext, tag = cipher.encrypt_and_digest(data.encode())
    return key, cipher.nonce, ciphertext, tag

def aes_decrypt(key, nonce, ciphertext, tag):
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    return cipher.decrypt_and_verify(ciphertext, tag).decode()

In [3]:
# --- RSA ---
def gen_rsa_key():
    key = RSA.generate(2048)
    return key, key.publickey()

def rsa_encrypt(pub, data):
    cipher = PKCS1_OAEP.new(pub)
    return cipher.encrypt(data.encode())

def rsa_decrypt(priv, ciphertext):
    cipher = PKCS1_OAEP.new(priv)
    return cipher.decrypt(ciphertext).decode()

In [4]:
# --- SIGNATURE ---
def sign(priv, message):
    h = SHA256.new(message.encode())
    return pss.new(priv).sign(h)

def verify(pub, message, signature):
    h = SHA256.new(message.encode())
    try:
        pss.new(pub).verify(h, signature)
        return True
    except:
        return False

In [5]:
# --- HASH ---
def hash_all(msg):
    return {
        "MD5": hashlib.md5(msg.encode()).hexdigest(),
        "SHA1": hashlib.sha1(msg.encode()).hexdigest(),
        "SHA256": hashlib.sha256(msg.encode()).hexdigest()
    }

In [6]:
# ------ TESTING ------
msg = "Hello Cryptography"

# AES
key, nonce, ct, tag = aes_encrypt(msg)
print("AES decrypted:", aes_decrypt(key, nonce, ct, tag))

# RSA
priv, pub = gen_rsa_key()
cipher = rsa_encrypt(pub, msg)
print("RSA decrypted:", rsa_decrypt(priv, cipher))

# HASH
print("Hashes:", hash_all(msg))

# SIGNATURE
signature = sign(priv, msg)
print("Signature valid:", verify(pub, msg, signature))

AES decrypted: Hello Cryptography
RSA decrypted: Hello Cryptography
Hashes: {'MD5': 'a66f551fec97316c7283d30bd04711d2', 'SHA1': '94c52d1ae5ab0ff70e16872e2aa11eef93bb4bf1', 'SHA256': 'c1b2a7351baf40d991b3a1d88e2e82c9036b92df830fb82ed70850b8d820d3b5'}
Signature valid: True
